In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


import pandas as pd
import numpy as np
import scipy as sp
import patsy
import pystan
#from pystan import StanFit4model

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()



import statsmodels.api as sm

import rpy2.robjects as ro
#import pandas.rpy.common as com

pd.set_option('max_columns', 50)






In [6]:

url = "http://stat.columbia.edu/~gelman/arm/examples/arsenic/wells.dat"
wells = pd.read_table(url, sep='\s+', header=0, index_col=0)
wells["dist100"] = wells['dist']/100

y = wells['switch']
wellsNp = wells.as_matrix()
y_mod = wellsNp[:, 0]

y_hat = y.transpose()

y_mean = np.mean(y_mod)

s = sp.special.logit(y_mean)

logit = np.log(y_mean / (1 - y_mean))

f = "~ 0 + dist + arsenic"
mat = patsy.dmatrix(f, wells, return_type='dataframe')
nrow, ncol = mat.shape


data_loo = {'N': nrow , 'P': ncol, 'y': y, 'x': mat, 'a': s }


In [7]:

logistic = '''
data {
  int<lower=0> N; 
  int<lower=0> P;
  int<lower=0,upper=1> y[N];
  matrix[N,P] x; 
  real a;
}
parameters {
  real beta0;
  vector[P] beta;
}
model {
  beta0 ~ student_t(7, a, 0.1);
  beta ~ student_t(7, 0, 1);
  y ~ bernoulli_logit(beta0 + x * beta);
}
generated quantities {
  vector[N] log_lik;
  for (n in 1:N)
    log_lik[n] <- bernoulli_logit_log(y[n], beta0 + x[n] * beta);
}
'''

In [13]:
fit1 = pystan.stan(model_code=logistic, data=data_loo, iter=1000, chains=4)

#print(fit1)

In [10]:
#log_like = pystan.stanfit4model
#print fit1

parameter_name = "log_lik"
type(fit1)
matrix = fit1.extract(pars=parameter_name)
logLik = matrix['log_lik']
#print(matrix['log_lik'])
#np.take(fit1, parameter_name)

[[-0.31205243 -0.7299264  -1.18988203 ..., -0.80826273 -0.76167722
  -0.61563212]
 [-0.33393141 -0.69647098 -1.15296022 ..., -0.86170177 -0.8042652
  -0.58091826]
 [-0.33215004 -0.77216172 -1.13833661 ..., -0.75068446 -0.71190467
  -0.66108946]
 ..., 
 [-0.32657014 -0.74721188 -1.15526254 ..., -0.78085149 -0.73858918
  -0.63644058]
 [-0.32295474 -0.73147656 -1.16616104 ..., -0.7951284  -0.75292151
  -0.62389538]
 [-0.32303773 -0.76167339 -1.16032185 ..., -0.79580786 -0.74196517
  -0.63212885]]


In [11]:
import sys
sys.path.insert(0, '/Users/detcitty/Documents/Stan/loo/loo/')

import loo



In [12]:
test = loo.psisloo(logLik)

/Users/detcitty/Documents/Stan/loo/loo/loo.py:230: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif sort == 'in-place':
/Users/detcitty/Documents/Stan/loo/loo/loo.py:248: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  bs /= 3 * x[sort[np.floor(n/4 + 0.5) - 1]]
